Goal of this notebook is to understand how to call cphf.solve_nos1(fvind, mo_energy, mo_occ, h1,max_cycle=20, tol=1e-9, hermi=False)  (better the shape of the arrays passed as arguments)

In [1]:
import pyscf
import pyscf.qmmm
from pyscf import gto, scf
import numpy as np
import matplotlib.pyplot as plt
import inspect
#from pyscf.geomopt.berny_solver import optimize
#from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
from pyscf import lib
import inspect
from functools import reduce
from pyscf.scf import cphf
angstrom = 1 / 0.52917721067
#from pyscf.scf._response_functions import _gen_rhf_response 

In [2]:
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV.reshape((1,dV.shape[0],dV.shape[1]))

def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

In [31]:
import time
from functools import reduce
import numpy
from pyscf import lib
from pyscf.lib import logger
from pyscf.scf import cphf
from pyscf.prop.nmr import rhf as rhf_nmr
#from pyscf.scf import _gen_rhf_response

def dipole(mf):
    return mf.dip_moment(mf.mol, mf.make_rdm1())


def polarizability(polobj, with_cphf=True):
    
    log = logger.new_logger(polobj)
    mf = polobj._scf
    mol = mf.mol
    mo_energy = mf.mo_energy
    mo_coeff = mf.mo_coeff
    mo_occ = mf.mo_occ
    occidx = mo_occ > 0
    orbo = mo_coeff[:, occidx]
    orbv = mo_coeff[:,~occidx]

    charges = mol.atom_charges()
    coords  = mol.atom_coords()
    charge_center = numpy.einsum('i,ix->x', charges, coords) / charges.sum()
    int_r=DeltaV(mol,[-.01,.01])
    h1 = lib.einsum('xpq,pi,qj->xij', int_r, mo_coeff.conj(), orbo) #going to molecular orbitals?
    print(h1.shape, "shape (1, n ao, nocc)" )
    #print(mo_energy, mo_occ,h1)
    s1 = numpy.zeros_like(h1)
    vind = polobj.gen_vind(mf, mo_coeff, mo_occ)
    if with_cphf:
        mo1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1,
                         polobj.max_cycle_cphf, polobj.conv_tol,
                         verbose=log)[0]
    else:
        mo1 = rhf_nmr._solve_mo1_uncoupled(mo_energy, mo_occ, h1, s1)[0]
    #e2 = numpy.einsum('xpi,ypi->xy', h1, mo1)
    # *-1 from the definition of dipole moment. *2 for double occupancy
    #e2 = (e2 + e2.T) * -2
    #return e2
    return mo1

def gen_vind(self, mf, mo_coeff, mo_occ):  #polarizability
        '''Induced potential'''
        vresp = mf.gen_response(hermi=1)
        occidx = mo_occ > 0
        orbo = mo_coeff[:, occidx]
        nocc = orbo.shape[1]
        nao, nmo = mo_coeff.shape
        def vind(mo1):
            dm1 = lib.einsum('xai,pa,qi->xpq', mo1.reshape(-1,nmo,nocc), mo_coeff,orbo.conj())
            dm1 = (dm1 + dm1.transpose(0,2,1).conj()) * 2
            v1mo = lib.einsum('xpq,pi,qj->xij', vresp(dm1), mo_coeff.conj(), orbo)
            return v1mo.ravel()
        return vind

In [35]:
mol = gto.M(atom='He 0 0 0; He 0 0 1.2', unit="Bohr",basis="3-21g")
mol=gto.Mole(atom="C 0 0 0 ;O 0 0 2.", unit="Bohr", basis= "sto-3G")
mf = scf.RHF(mol)
e=mf.scf()

Initialize <pyscf.gto.mole.Mole object at 0x7fe059951150> in <pyscf.scf.hf.RHF object at 0x7fe0580214d0>


converged SCF energy = -111.199724042754


In [36]:
from pyscf.prop.polarizability.rhf import Polarizability
plo=Polarizability(mf) #polariz obj

In [37]:
polarizability(plo)

(1, 10, 7) shape (1, n ao, nocc)


array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00

In [10]:
vind=plo.gen_vind(mf, mf.mo_coeff,mf.mo_occ)

NameError: name 'dVf' is not defined